在计算机视觉的实际应用中，分类是远远不够的，更多的场景需要能同时探测多个物体，不仅包括这些物体的分类，而且包括这些物体的位置和尺寸。通常使用包围盒（Bounding Box）来表示这些物体的位置和尺寸。

![img](images/chapter15/object_detection.png)



目标检测任务通常使用 IoU（Intersection over Union） 和 mAP 作为模型评价指标。IoU 其定义为模型产生的目标窗口和原来标记窗口的交叠率，即检测结果（预测生成的包围盒）与真实值（Ground Truth）的交集比上它们的并集。因此 IoU 取值范围为 0~1，数值越高代表重叠程度越高。

![img](images/chapter15/IoU.png)

经典的使用神经网络实现目标检测的模型有：

- [R-CNN](https://arxiv.org/abs/1311.2524)
- [SPPNet](https://arxiv.org/abs/1406.4729)
- [Fast R-CNN](https://arxiv.org/abs/1504.08083)
- [Faster R-CNN](https://arxiv.org/abs/1506.01497)
- [SSD](https://arxiv.org/abs/1512.02325)
- [YOLO](https://arxiv.org/abs/1506.02640)

# 1. [R-CNN](https://arxiv.org/abs/1311.2524)

R-CNN（Region-based Convolutional Network） 模型发展到现在已经成为一个系列，从最初的 R-CNN 模型发展到 Fast R-CNN 模型，之后在 2015 年再一次突破，提出了 Faster R-CNN模型。近几年来，基于深度学习的目标检测方法发展很快，相继提出了很多精妙的思想，R-CNN 模型系列功不可没，该模型是很多算法的基础思想。

R-CNN 结合了 [Selective Search](https://www.cs.mcgill.ca/~hsyed2/selectivesearch/Object-Recognition/) 算法的候选区域和由卷积神经网络产生的视觉丰富特征，它的性能比当时最好的检测模型在 Pascal VOC 2012 数据集上有将近 30% 的性能提升，mAP（mean average precision）指标从 40.9% 提升到了 53.3%。

R-CNN 模型的整体框架为：

- 采用 Selective Search 算法生成候选区域（region proposals）
      每张图片大约生成 2000 个候选区域，由于候选区域尺寸不一，所以通过拉伸变换，形成统一的输入尺寸 227×227。

- 运用 CNN 网络进行特征提取
      将统一尺寸的候选区域图像输入 AlexNet 网络并提取 4096 维特征向量
      
- 对候选区域图像进行分类
      使用 SVM 分类器对 CNN 输出特征进行分类
      
- 包围盒回归（Bounding Box Regression）
      用 Bounding Box 回归值校正原来的候选区域窗口，生成预测窗口坐标

![img](images/chapter15/R-CNN.png)

R-CNN 的训练步骤分为四步：

- 在 ImageNet 数据集上进行预训练（pre-training）
![img](images/chapter15/R-CNN_pre_train.png)


- 利用成熟的权重参数在 PASCAL VOC 数据集上进行微调（fine-tune）
      将网络最后一个 1000 个神经元的全连接层改为 21 个神经元
      
![img](images/chapter15/R-CNN_fine_tune.png)
      
- 使用 CNN 提取的特征作为输入信息，为每个分类训练一个 SVM，可以将精度再提高 3.3%

![img](images/chapter15/R-CNN_train_SVM.png)

- 包围盒回归训练

将包围盒回归训练的输入数据记做 $\{(P^i,G^i)\}_{i=1,...,N}$，其中
$$ P^i=(P_x^i, P_y^i, P_w^i, P_h^i) $$
$$ G^i=(G_x^i, G_y^i, G_w^i, G_h^i) $$

$P_x^i$ 和 $P_y^i$ 代表包围盒的中心坐标，$P_w^i$ 和 $P_h^i$ 分别代表包围盒的宽度和高度。为了简化说明，下面省略 $P$ 和 $G$ 的上标 $i$。因此，回归任务的目标即是，给定 $(Px,Py,Pw,Ph)$ 寻找一种映射 $f$， 使得$f(P_x,P_y,P_w,P_h)=(\hat G_x,\hat G_y,\hat G_w,\hat G_h)$，并且 $ (\hat G_x,\hat G_y,\hat G_w,\hat G_h) \approx (G_x,G_y,G_w,G_h)$

$$\hat G_x=P_w d_x(P)+P_x$$
$$\hat G_y=P_h d_y(P)+P_y$$

$$\hat G_w=P_w \exp(d_w(P))$$
$$\hat G_h=P_h \exp(d_h(P))$$


包围盒回归学习任务就是学习 $d_x(P)$，$d_y(P)$，$d_w(P)$ 和 $d_h(P)$这四个变换。

在测试阶段，R-CNN 使用训练好的 CNN 和 SVM 对每张图像的约 2000 个候选区域进行分类并排序，再使用 **非极大值抑制算法**（non-maximum suppression）去除重叠区域，得到最终预测结果。

R-CNN 模型由于提出较早，相比于目前的目标检测网络模型，精确度不够高，速度也较慢，无法满足实时系统需要。但 R-CNN 让业界第一次看到了深度学习在目标检测上的力量和潜力，是深度学习在目标检测任务上迈出的重要一步。

# 2. [Fast R-CNN](https://arxiv.org/abs/1504.08083)

R-CNN 模型主要存在以下三个问题：

- 测试速度慢

      R-CNN 将一张图像分解成近 2000 个候选区域，每个候选区域拉伸形成的图像都会单独通过 CNN 提取特征。实际上这些候选区域之间大量重叠，特征值之间完全可以共享，因此造成了运算能力的浪费
      
- 训练速度慢
      R-CNN 在训练时，在应用 SVM 分类之前，将 CNN 提取的特征存储在硬盘上，在硬盘上大量读写交换数据造成训练速度缓慢
      
- 训练所需空间大
      R-CNN 中独立的 SVM 分类器和回归器需要大量特征作为训练样本，需要存储上千GB的数据

R-CNN 的原作者 Ross Girshick 在吸收了 [SPPNet](https://arxiv.org/abs/1406.4729) 的优点后再一次推出了 R-CNN 的继承者 Fast R-CNN。Fast R-CNN 构思精巧，流程更为紧凑，大幅提升了目标检测的速度。与 R-CNN 相比，训练时间从 84 小时减少为 9.5 小时，测试时间从 47 秒减少为 0.32 秒，在 PASCAL VOC 2007 数据集上的准确率为 66%。

Fast R-CNN 模型整体训练框架大致为：

- 将整张图像输入至 CNN 网络，进行特征提取


- 使用 Selective Search 算法生成候选区域，并将候选区域窗口映射到 CNN 的最后一层卷积特征图上


- 通过 ROI（Regions of Interest） Pooling 层将每个候选区域窗口生成固定尺寸的特征图


- 将特征图输入至全连接层网络提取 ROI 特征向量


- Softmax 层对 ROI 特征向量进行分类，回归层使用 Smooth L1 损失函数进行包围盒回归，并加权联合二者损失函数值

![img](images/chapter15/Fast_R-CNN.png)

由于用于目标检测的数据集较小，无法充分从零开始训练网络，Fast R-CNN 和 R-CNN 一样采用在预训练网络模型上微调参数的方法，即通过 ImageNet 数据集对 CNN 进行预训练，然后用 PASCAL VOC 数据集再进行微调训练。Fast R-CNN 使用了三种预训练网络模型，分别是小型网络 CaffeNet（AlexNet 修改版）、中型网络 VGG_VNN_M_1024（和 AlexNet 同样深度的 VGG 网络）和大型网络 VGG16。

另外，Fast R-CNN 验证了使用 Softmax 层可以实现比 SVM 更好的分类性能：

|模型|分类器|CaffeNet|VGG_VNN_M_1024|VGG16|
|---|----|---|--|--|
|Fast R-CNN|SVM|56.3|58.7|66.8|
|Fast R-CNN|Softmax|**57.1**|**59.2**|**66.9**|

# 3. [Faster R-CNN](https://arxiv.org/abs/1506.01497)

虽然 Fast R-CNN 基于 R-CNN 有很大的改进，但 Fast R-CNN 仍然基于 Selective Search 算法提取候选区域，而 Selective Search 算法提取候选区域是基于 CPU 运行的，无法借用 GPU 的高度并行运算能力，因此效率依然不高。另外，这种方法为每幅图像提供大约 2000 个候选区域，如此多的候选区域加重了后面神经网络的计算压力。

Faster R-CNN 在吸取了Fast R-CNN 特点的前提下，将目标检测的四个基本步骤（候选区域生成、特征提取、分类和包围盒回归）统一到一个深度神经网络框架之内。Faster R-CNN 加入了提取候选区域的神经网络 **RPN**（Region Proposal Network），用 RPN 直接预测出候选区域框，数据限定在 300 个。RPN 的预测在 GPU 中完成，并且卷积网络和 Faster R-CNN 部分共享，因此大幅提升了目标检测的速度。

Faster R-CNN 模型的整体框架如下：

- 将整张图像输入 CNN 进行特征提取


- 使用 RPN 为每张图像生成 300 个候选区域


- 将候选区域映射到 CNN 的最后一层卷积特征图上


- 通过 ROI Pooling 层将每个候选区域特征图变成固定尺寸的特征图


- 使用 Softmax 交叉熵损失和 Smooth L1 损失对分类概率和包围盒回归联合训练

在生成候选区域的 RPN 设计中，Faster R-CNN 在特征图上运用滑动窗口。为了能够应对不同尺寸的物体，Faster R-CNN 采用了 3 中不同类型的滑动窗口，长宽比分别是 1:1，2:1 和 1:2，并用 3 个尺度缩放滑动窗口，因此共采用了 9 种类型的滑动窗口。这些窗口经过卷积形成 256 维向量，最终通过分类挑选出 300 个得分最高的窗口作为最终的候选区域。

![img](images/chapter15/region_proposal_network.png)

Faster R-CNN 与 Fast R-CNN 性能对比：

|模型|网络|卷积运算|候选区域生成|逐区域|合计|速度|
|---|---|----|----|----|---|---|
|Fast R-CNN|VGG16|146 ms|1510 ms|174 ms|1830 ms| 0.5 fps|
|Faster R-CNN|VGG16|141 ms|10 ms|47 ms|198 ms|5 fps|

Faster R-CNN 是深度学习算法在目标检测任务上的重大进步，让业界看到了深度学习算法在目标检测任务上进行 **实时处理** 的希望。

# 4. [YOLO](https://arxiv.org/abs/1506.02640)

2015年，随着 YOLO 算法的出现，深度学习目标检测算法开始有了两步（two-stage）和单步（single-stage）之分。区别于以 R-CNN 系列为代表的两步检测算法，YOLO舍去了候选区域提取分支，直接将特征提取、候选框回归和分类在同一个无分支的卷积网络中完成，使得网络结构变得简单，检测速度较 Faster R-CNN 也有近 10 倍的提升，这使得深度学习目标检测算法开始能够满足实时检测任务的需求。